In [1]:
from ilqr import ILQR
import gym
import sys
import env
import time
# import jax.numpy as np
from tqdm.notebook import trange

In [12]:
env = gym.make('Humanoid-v3')
obs = env.reset()


def final_cost(x):
    return 0.5 * (np.square(1.0 - np.cos(x[2])) + np.square(x[1]) + np.square(x[3]))


def running_cost(x, u):
    return 0.5 * np.sum(np.square(u))


def model(x, u):
    return env._state_eq(x, u)


cnt = 0
t = time.process_time()
fps = []
while True:
#     env.render()
    obs, _, _, _ = env.step(env.action_space.sample())
    cnt += 1
    if time.process_time() - t >= 1:
        fps.append(cnt)
        print(cnt)
        cnt = 0
        t = time.process_time()

1696
1675
1669


KeyboardInterrupt: 

In [17]:
env.action_space

Box(17,)

In [18]:
env.observation_space

Box(376,)